In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('/opt/psegs')

from psegs.datasets import ios_lidar

base_dir = '/outer_root/home/au/lidarphone_scans/2021_06_27_12_37_38'
# base_dir = '/outer_root/home/au/lidarphone_scans/landscape_home_button_right_07_09_49'

from oarphpy import util as oputil
json_paths = oputil.all_files_recursive(base_dir, pattern='frame*.json')
json_paths = sorted(json_paths)
cis = [ios_lidar.threeDScannerApp_create_camera_image(p) for p in json_paths]

print(len(cis))
cis[0]
    

149


CameraImage(sensor_name='camera_front', image_jpeg=bytearray(b''), image_png=bytearray(b''), image_factory=CloudpickeledCallable(_func_pyclass=psegs.datasets.ios_lidar.<lambda>), width=1920, height=1440, timestamp=853720759521708, ego_pose=Transform(rotation=array([[-0.16402921,  0.36367464, -0.91697067],
       [-0.04168281,  0.92617565,  0.37478161],
       [ 0.98557436,  0.09969707, -0.13676088]]), translation=array([[-2.92258954],
       [ 0.93101001],
       [ 0.19710636]]), src_frame='ego', dest_frame='world'), ego_to_sensor=Transform(rotation=array([[ 1,  0,  0],
       [ 0, -1,  0],
       [ 0,  0, -1]]), translation=array([[0.],
       [0.],
       [0.]]), src_frame='camera_front', dest_frame='ego'), K=array([[1.44565613e+03, 0.00000000e+00, 9.69324890e+02],
       [0.00000000e+00, 1.44565613e+03, 6.90660400e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), extra={'threeDScannerApp.cameraGrain': '0', 'threeDScannerApp.time': '853720.7595217085', 'threeDScannerAp

In [ ]:
# from psegs.mesh2rgb import pytorch3d_camera_images_to_rgbd_debug

# mesh_path = os.path.join(base_dir, 'export_refined.obj')
# outpath = os.path.join(base_dir, 'pytorch3d_rgbd_debug.mp4')

# pytorch3d_camera_images_to_rgbd_debug(cis, )

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=False)         # initiate notebook for offline plot

In [ ]:
import os
import numpy as np
import open3d as o3d
mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export_refined.obj'))
vertices = np.asarray(mesh.vertices)
print(vertices.shape)

import numpy as np
v_sub = vertices[np.random.choice(vertices.shape[0], 40000, replace=False)]
print(v_sub.shape)

In [ ]:
import plotly
import plotly.graph_objects as go

plots = [ci.to_plotly_world_frame_3d() for ci in cis[:1]]

import pandas as pd
cloud_df = pd.DataFrame(v_sub, columns=['x', 'y', 'z'])
from psegs.util.plotting import rgb_for_distance
cloud_df['color'] = [
  rgb_for_distance(np.linalg.norm(pt), period_meters=1.)
  for pt in cloud_df[['x', 'y', 'z']].values
]
scatter = go.Scatter3d(
                x=cloud_df['x'], y=cloud_df['y'], z=cloud_df['z'],
                mode='markers',
                marker=dict(size=2, color=cloud_df['color'], opacity=0.5),)

plots.append(scatter)


fig = go.Figure(data=plots)

fig.update_layout(
  width=1000, height=700,
  scene_aspectmode='data')
  # scene_camera=dict(
  #   up=dict(x=0, y=0, z=1),
  #   eye=dict(x=0, y=0, z=0),
  #   center=dict(x=1, y=0, z=0),
  # ))
    
iplot(fig)
    
    
# plot_str = plotly.offline.plot(fig, output_type='div')

# html += '<br/><br/>' + plot_str




In [ ]:
assert False
# # https://chart-studio.plotly.com/~empet/15040/plotly-mesh3d-from-a-wavefront-obj-f/#/


# def obj_data_to_mesh3d(odata):
#     # odata is the string read from an obj file
#     vertices = []
#     faces = []
#     lines = odata.splitlines()   
   
#     for line in lines:
#         slist = line.split()
#         if slist:
#             if slist[0] == 'v':
#                 vertex = np.array(slist[1:], dtype=float)
#                 vertices.append(vertex)
#             elif slist[0] == 'f':
#                 face = []
#                 for k in range(1, len(slist)):
#                     face.append([int(s) for s in slist[k].replace('//','/').split('/')])
#                 if len(face) > 3: # triangulate the n-polyonal face, n>3
#                     faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
#                 else:    
#                     faces.append([face[j][0]-1 for j in range(len(face))])
#             else: pass
    
    
#     return np.array(vertices), np.array(faces)

# import os
# with open(os.path.join(base_dir, 'export_refined.obj'), 'rb') as f:
#     obj_data = f.read().decode('utf-8')
# vertices, faces = obj_data_to_mesh3d(obj_data)
# print('vertices.shape', vertices.shape)
# print('faces.shape', faces.shape)


# x, y, z = vertices[:,:3].T
# I, J, K = faces.T

# mesh = go.Mesh3d(
#             x=-x,
#             y=-y,
#             z=z,
# #             vertexcolor=vertices[:, 3:], #the color codes must be triplets of floats  in [0,1]!!                      
#             i=I,
#             j=J,
#             k=K,
#             name='',
#             showscale=False)

# layout = go.Layout(width=900,
#                    height=800,
#                    scene=dict(xaxis=dict(visible=False),
#                               yaxis=dict(visible=False),  
#                               zaxis=dict(visible=False), 
#                               aspectratio=dict(x=1.5,
#                                                y=0.9,
#                                                z=0.5
#                                          ),
#                               camera=dict(eye=dict(x=1., y=1., z=0.5)),
#                         ),
#                   ) 

# fig = go.Figure(data=[mesh], layout=layout)
# iplot(fig)

In [ ]:
# import open3d as o3d
# mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export_refined.obj'))
# vertices = np.asarray(mesh.vertices)
# print(vertices.shape)

# import numpy as np
# v_sub = vertices[np.random.choice(vertices.shape[0], 10000, replace=False)]
# print(v_sub.shape)

In [ ]:
cloud_raw = vertices

import imageio
writer = imageio.get_writer('/outer_root/home/au/lidarphone_scans/test_video.mp4', fps=5)

for i in range(len(cis)):
    cloud_ego = cis[i].ego_pose.get_inverse().apply(cloud_raw).T

    from psegs import datum
    pc = datum.PointCloud(cloud=cloud_ego)

    debug = cis[i].get_debug_image(clouds=[pc], period_meters=0.1)
    writer.append_data(debug)
    print(i)
writer.close()

#     from io import BytesIO
#     import IPython.display
#     import numpy as np
#     import PIL.Image
#     def showarray(a, fmt='png'):
#         a = np.uint8(a)
#         f = BytesIO()
#         PIL.Image.fromarray(a).save(f, fmt)
#         IPython.display.display(IPython.display.Image(data=f.getvalue()))

#     showarray(debug)

In [ ]:
import sys
sys.path.append('/opt/psegs')

import os

ROOT = '/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/'

for d in os.listdir(ROOT):
    if '.DS_Store' in d:
        continue
    base_dir = os.path.join(ROOT, d)
    print(base_dir)
    

    from psegs.datasets import ios_lidar


    from oarphpy import util as oputil
    json_paths = oputil.all_files_recursive(base_dir, pattern='frame*.json')
    json_paths = sorted(json_paths)
    
    try:
        cis = [ios_lidar.threeDScannerApp_create_camera_image(p) for p in json_paths]
    except AssertionError as e:
        continue

    print(len(cis))
    
    import os
    import numpy as np
    import open3d as o3d
    mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export.obj'))
    vertices = np.asarray(mesh.vertices)
    print(vertices.shape)

    cloud_raw = vertices

    outpath = os.path.join(ROOT, d + '.mp4')
    
    import imageio
    writer = imageio.get_writer(outpath, fps=5)

    for i in range(len(cis)):
        cloud_ego = cis[i].ego_pose.get_inverse().apply(cloud_raw).T
        cloud_ego[:, 0] *= -1

        from psegs import datum
        pc = datum.PointCloud(cloud=cloud_ego)

        debug = cis[i].get_debug_image(clouds=[pc], period_meters=0.1)
        writer.append_data(debug)
        print(i)
    writer.close()
    
    print('done', outpath)


In [ ]:
# # !apt-get install -y python3-pybind11
# # !cd /opt && git clone https://github.com/NVIDIA/cub
# # !CUB_HOME=/opt/cub pip3 install "git+https://github.com/facebookresearch/pytorch3d.git@stable"

# # import os
# # !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
# # !tar xzf 1.10.0.tar.gz
# # os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
# # !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'`


# # !pip3 install torch torchvision torchaudio
# # !pip3 install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu102_pyt190/download.html

# import os
# import sys
# import torch

# import pytorch3d

# import os
# import torch
# import matplotlib.pyplot as plt

# from pytorch3d.utils import ico_sphere
# import numpy as np
# from tqdm.notebook import tqdm

# # Util function for loading meshes
# from pytorch3d.io import load_objs_as_meshes, save_obj

# from pytorch3d.loss import (
#     chamfer_distance, 
#     mesh_edge_loss, 
#     mesh_laplacian_smoothing, 
#     mesh_normal_consistency,
# )

# # Data structures and functions for rendering
# from pytorch3d.structures import Meshes
# from pytorch3d.renderer import (
#     look_at_view_transform,
#     OpenGLPerspectiveCameras, 
#     PointLights, 
#     DirectionalLights, 
#     Materials, 
#     RasterizationSettings, 
#     MeshRenderer, 
#     MeshRasterizer,  
#     SoftPhongShader,
#     SoftSilhouetteShader,
#     SoftPhongShader,
#     TexturesVertex
# )

# # add path for demo utils functions 
# import sys
# import os
# sys.path.append(os.path.abspath(''))

# !wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/docs/tutorials/utils/plot_image_grid.py
# from plot_image_grid import image_grid

# !mkdir -p data/cow_mesh
# !wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj
# !wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl
# !wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png
    

    
# if torch.cuda.is_available():
#     device = torch.device("cuda:0")
#     torch.cuda.set_device(device)
# else:
#     device = torch.device("cpu")

# # Set paths
# DATA_DIR = "./data"
# obj_filename = os.path.join(DATA_DIR, "cow_mesh/cow.obj")

# # Load obj file
# mesh = load_objs_as_meshes([obj_filename], device=device)

# # We scale normalize and center the target mesh to fit in a sphere of radius 1 
# # centered at (0,0,0). (scale, center) will be used to bring the predicted mesh 
# # to its original center and scale.  Note that normalizing the target mesh, 
# # speeds up the optimization but is not necessary!
# verts = mesh.verts_packed()
# N = verts.shape[0]
# center = verts.mean(0)
# scale = max((verts - center).abs().max(0)[0])
# mesh.offset_verts_(-center)
# mesh.scale_verts_((1.0 / float(scale)))


# # the number of different viewpoints from which we want to render the mesh.
# num_views = 20

# # Get a batch of viewing angles. 
# elev = torch.linspace(0, 360, num_views)
# azim = torch.linspace(-180, 180, num_views)

# # Place a point light in front of the object. As mentioned above, the front of 
# # the cow is facing the -z direction. 
# lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

# # Initialize an OpenGL perspective camera that represents a batch of different 
# # viewing angles. All the cameras helper methods support mixed type inputs and 
# # broadcasting. So we can view the camera from the a distance of dist=2.7, and 
# # then specify elevation and azimuth angles for each viewpoint as tensors. 
# R, T = look_at_view_transform(dist=2.7, elev=elev, azim=azim)
# cameras = OpenGLPerspectiveCameras(device=device, R=R, T=T)

# # We arbitrarily choose one particular view that will be used to visualize 
# # results
# camera = OpenGLPerspectiveCameras(device=device, R=R[None, 1, ...], 
#                                   T=T[None, 1, ...]) 

# # Define the settings for rasterization and shading. Here we set the output 
# # image to be of size 128X128. As we are rendering images for visualization 
# # purposes only we will set faces_per_pixel=1 and blur_radius=0.0. Refer to 
# # rasterize_meshes.py for explanations of these parameters.  We also leave 
# # bin_size and max_faces_per_bin to their default values of None, which sets 
# # their values using heuristics and ensures that the faster coarse-to-fine 
# # rasterization method is used.  Refer to docs/notes/renderer.md for an 
# # explanation of the difference between naive and coarse-to-fine rasterization. 
# raster_settings = RasterizationSettings(
#     image_size=128, 
#     blur_radius=0.0, 
#     faces_per_pixel=1, 
# )

# # Create a Phong renderer by composing a rasterizer and a shader. The textured 
# # Phong shader will interpolate the texture uv coordinates for each vertex, 
# # sample from a texture image and apply the Phong lighting model
# renderer = MeshRenderer(
#     rasterizer=MeshRasterizer(
#         cameras=camera, 
#         raster_settings=raster_settings
#     ),
#     shader=SoftPhongShader(
#         device=device, 
#         cameras=camera,
#         lights=lights
#     )
# )

# # Create a batch of meshes by repeating the cow mesh and associated textures. 
# # Meshes has a useful `extend` method which allows us do this very easily. 
# # This also extends the textures. 
# meshes = mesh.extend(num_views)

# # Render the cow mesh from each viewing angle
# target_images = renderer(meshes, cameras=cameras, lights=lights)

# # Our multi-view cow dataset will be represented by these 2 lists of tensors,
# # each of length num_views.
# target_rgb = [target_images[i, ..., :3] for i in range(num_views)]
# target_cameras = [OpenGLPerspectiveCameras(device=device, R=R[None, i, ...], 
#                                            T=T[None, i, ...]) for i in range(num_views)]

# image_grid(target_images.cpu().numpy(), rows=4, cols=5, rgb=True)
# plt.show()

In [ ]:
# from pytorch3d.renderer.mesh.rasterizer import MeshRasterizer

# from torchvision.transforms import ToPILImage

# to_img = ToPILImage()

# rasterizer = MeshRasterizer(
#     cameras=cameras, 
#     raster_settings=raster_settings
# )

# fragments = rasterizer(meshes)

# to_img(fragments.zbuf.cpu().squeeze())

In [ ]:
# import numpy as np
# IOS_CAM_TO_PYTORCH = np.array([
#     [ 0, -1,  0,  0],
#     [ 0,  0,  1,  0],
#     [-1,  0,  0,  0],
#     [ 0,  0,  0,  1],
#   ], dtype=np.float32)

from pytorch3d.io import load_obj
obj_path = os.path.join(base_dir, 'export_refined.obj')
verts, faces_idx, _ = load_obj(obj_path)
faces = faces_idx.verts_idx
print('verts', verts.shape)
print('faces', faces.shape)

nverts = verts.numpy()
nfaces = faces.numpy()

In [ ]:

CI = cis[130]

import numpy as np
# obj_path = os.path.join(base_dir, 'export_refined.obj')
fov_x, fov_y = CI.get_fov()
K = CI.K
height, width = CI.height, CI.width
pose = CI.ego_pose['ego', 'world'].get_inverse().get_transformation_matrix(homogeneous=True)
pose = pose.astype(np.float32)
K = K.astype(np.float32)



import numpy as np
world2pytorch = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1],
], dtype=np.float32)

pose = world2pytorch @ pose

# pose[0, 0] *= -1
# pose[1, 1] *= -1
# pose[2, 2] *= -1

import os
import sys
import torch

import pytorch3d

import os
import torch
import matplotlib.pyplot as plt

from pytorch3d.utils import ico_sphere
import numpy as np
from tqdm.notebook import tqdm




# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, save_obj, load_obj

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader,
    TexturesVertex,
    AmbientLights
)

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))


# io utils
from pytorch3d.io import load_obj

# datastructures
from pytorch3d.structures import Meshes

# 3D transformations functions
from pytorch3d.transforms import Rotate, Translate

# rendering components
from pytorch3d.renderer import (
    FoVPerspectiveCameras, look_at_view_transform, look_at_rotation, 
    RasterizationSettings, MeshRenderer, MeshRasterizer, BlendParams,
    SoftSilhouetteShader, HardPhongShader, PointLights, TexturesVertex,
    HardGouraudShader, SoftGouraudShader,HardFlatShader,PerspectiveCameras,FoVOrthographicCameras,
)


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
# device = torch.device("cpu")

# # Set paths
# DATA_DIR = "./data"
# obj_filename = os.path.join(DATA_DIR, "cow_mesh/cow.obj")

# Load obj file
# mesh = load_objs_as_meshes([obj_path], device=device)

# Load the obj and ignore the textures and materials.
# verts, faces_idx, _ = load_obj(obj_path)
# faces = faces_idx.verts_idx
# print('verts', verts.shape)
# print('faces', faces.shape)
import torch
verts = torch.from_numpy(nverts)
faces = torch.from_numpy(nfaces)

# Initialize each vertex to be white in color.
verts_rgb = .9 * torch.ones_like(verts)[None]  # (1, V, 3)
textures = TexturesVertex(verts_features=verts_rgb.to(device))

# Create a Meshes object for the teapot. Here we have only one mesh in the batch.
teapot_mesh = Meshes(
    verts=[verts.to(device)],   
    faces=[faces.to(device)], 
    textures=textures,
)
print('teapot_mesh', teapot_mesh)
# teapot_mesh = load_objs_as_meshes([obj_path], device=device)


import torch
import numpy as np
R = torch.from_numpy(pose[:3, :3].reshape([1, 3, 3])).to(device)
T = torch.from_numpy(pose[:3, 3].reshape([1, 3])).to(device)


# Select the viewpoint using spherical angles  
distance = 5   # distance from camera to the object
elevation = 50.0   # angle of elevation in degrees
azimuth = 0.0  # No rotation so the camera is positioned on the +Z axis. 

# Get the position of the camera based on the spherical angles
# R, T = look_at_view_transform(distance, elevation, azimuth, device=device)
print('R', R)
print('T', T)

tK = np.eye(4).astype(np.float32)
tK[:3, :3] = K

# # Great job pytorch3d!! 
# # https://github.com/facebookresearch/pytorch3d/blob/103da63393d6bbb697835ddbfc86b07572ea4d0c/tests/test_camera_conversions.py#L116
# tK[0, 0] = 1.1 * K[0, 0]
# tK[1, 1] = 1.1 * K[1, 1]
# tK[2, 0] = 1.1 * K[2, 0]
# tK[2, 1] = 1.1 * K[2, 1]


tK = torch.from_numpy(tK.reshape([1, 4, 4])).to(device)
print('K', tK)

image_size = torch.from_numpy(np.array([height, width]).reshape([1, 2])).to(device)
print('image_size', image_size)

# https://github.com/facebookresearch/pytorch3d/issues/522
from pytorch3d.utils.camera_conversions import cameras_from_opencv_projection




cameras = cameras_from_opencv_projection(R, T, tK, image_size, device=device).cuda()

# assert False, (cameras.R, cameras.T, cameras.get_world_to_view_transform().device)
# assert False, cameras.get_world_to_view_transform().device
# print('get_world_to_view_transform', xform.device, cameras.R, cameras.T)

# cameras = FoVPerspectiveCameras(device=device, fov=fov_x, degrees=False)#, K=K)

# # hack up cameras_from_opencv_projection
# camera_matrix = tK
tvec = T
# focal_length = torch.stack([camera_matrix[:, 0, 0], camera_matrix[:, 1, 1]], dim=-1)
# principal_point = camera_matrix[:, :2, 2]

# # Retype the image_size correctly and flip to width, height.
# image_size_wh = image_size.to(R).flip(dims=(1,))

# # Get the PyTorch3D focal length and principal point.
# focal_pytorch3d = focal_length / (0.5 * image_size_wh)
# p0_pytorch3d = -(principal_point / (0.5 * image_size_wh) - 1)

# For R, T we flip x, y axes (opencv screen space has an opposite
# orientation of screen axes).
# We also transpose R (opencv multiplies points from the opposite=left side).
R_pytorch3d = R.clone().permute(0, 2, 1)
T_pytorch3d = tvec.clone()
R_pytorch3d[:, :, :2] *= -1
T_pytorch3d[:, :2] *= -1
# cameras = PerspectiveCameras(
#             device=device, R=R_pytorch3d,
#             T=T_pytorch3d,
#             focal_length=focal_pytorch3d,
#             principal_point=p0_pytorch3d, image_size=image_size, in_ndc=True)

fov_x, fov_y = CI.get_fov()
cameras = FoVPerspectiveCameras(
    device=device, fov=fov_y, degrees=False, R=R_pytorch3d, T=T_pytorch3d, aspect_ratio=1.0)


# cameras = PerspectiveCameras(device=device, K=K, R=R, T=T, in_ndc=False, image_size=image_size)


raster_settings = RasterizationSettings(
    image_size=(height, width), 
    faces_per_pixel=1, 
)
lights = PointLights(
    device=device, 
    location=cameras.get_world_to_view_transform().transform_points(torch.tensor([[0., 0., -1.]]).cuda()),
)
# lights = AmbientLights(device=device)
blend_params = BlendParams(sigma=1e-4, gamma=1e-4, background_color=(0.1, 0.1, 0.1))
rasterizer = MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    )
phong_renderer = MeshRenderer(
    rasterizer=rasterizer,
    shader=HardPhongShader(device=device, cameras=cameras, lights=lights, blend_params=blend_params)
)


image_ref = phong_renderer(meshes_world=teapot_mesh)


import torchvision.transforms.functional as F
import numpy as np
pil_img = F.to_pil_image((255.0*image_ref.cpu().numpy()[0]).astype(np.uint8))

from IPython.display import display
display(pil_img)


# from pytorch3d.vis.plotly_vis import plot_batch_individually
# fig = plot_batch_individually([teapot_mesh, cameras])

# from plotly.graph_objs import *
# fig.layout = Layout(showlegend=True)
# fig.show()







# # Place a point light in front of the object. As mentioned above, the front of 
# # the cow is facing the -z direction. 
# lights = DirectionalLights()

# # Initialize an OpenGL perspective camera that represents a batch of different 
# # viewing angles. All the cameras helper methods support mixed type inputs and 
# # broadcasting. So we can view the camera from the a distance of dist=2.7, and 
# # then specify elevation and azimuth angles for each viewpoint as tensors. 

# # cameras = OpenGLPerspectiveCameras(device=device, R=pose[:3, :3], T=pose[:3, 3])

# # We arbitrarily choose one particular view that will be used to visualize 
# # results
# camera = OpenGLPerspectiveCameras(device=device, R=pose[:3, :3], T=pose[:3, 3])

# # Define the settings for rasterization and shading. Here we set the output 
# # image to be of size 128X128. As we are rendering images for visualization 
# # purposes only we will set faces_per_pixel=1 and blur_radius=0.0. Refer to 
# # rasterize_meshes.py for explanations of these parameters.  We also leave 
# # bin_size and max_faces_per_bin to their default values of None, which sets 
# # their values using heuristics and ensures that the faster coarse-to-fine 
# # rasterization method is used.  Refer to docs/notes/renderer.md for an 
# # explanation of the difference between naive and coarse-to-fine rasterization. 
# raster_settings = RasterizationSettings(
#     image_size=512, 
#     blur_radius=0.0, 
#     faces_per_pixel=1, 
# )

# # Create a Phong renderer by composing a rasterizer and a shader. The textured 
# # Phong shader will interpolate the texture uv coordinates for each vertex, 
# # sample from a texture image and apply the Phong lighting model
# renderer = MeshRenderer(
#     rasterizer=MeshRasterizer(
#         cameras=camera, 
#         raster_settings=raster_settings
#     ),
#     shader=SoftPhongShader(
#         device=device, 
#         cameras=camera,
#         lights=lights
#     )
# )

# # # Create a batch of meshes by repeating the cow mesh and associated textures. 
# # # Meshes has a useful `extend` method which allows us do this very easily. 
# # # This also extends the textures. 
# # meshes = mesh.extend(num_views)

# # Render the cow mesh from each viewing angle
# target_images = renderer([mesh], cameras=[camera], lights=lights)


# target_images
# # # Our multi-view cow dataset will be represented by these 2 lists of tensors,
# # # each of length num_views.
# # target_rgb = [target_images[i, ..., :3] for i in range(num_views)]
# # target_cameras = [OpenGLPerspectiveCameras(device=device, R=R[None, i, ...], 
# #                                            T=T[None, i, ...]) for i in range(num_views)]

# # image_grid(target_images.cpu().numpy(), rows=4, cols=5, rgb=True)
# # plt.show()

In [ ]:
pose

In [ ]:
import matplotlib.pyplot as plt

fragments = rasterizer(meshes_world=teapot_mesh)

zbuf = fragments.zbuf
plt.imshow(zbuf[0, ..., 0].cpu().numpy())
plt.show()
print('zbuf', zbuf.min(), zbuf.max(), zbuf[zbuf > 0].min())
# display(F.to_pil_image(image_ref.cpu().numpy()[0].astype(np.uint8)[:, :, -1]))
# image_ref.cpu().numpy()[0]

In [ ]:
zbuf[0, ..., 0].cpu().numpy().shape

In [ ]:
debug = CI.image
depth = zbuf[0, ..., 0].cpu().numpy()
h, w = debug.shape[:2]
px_y = np.tile(np.arange(h)[:, np.newaxis], [1, w])
px_x = np.tile(np.arange(w)[np.newaxis, :], [h, 1])
pyx = np.concatenate([px_y[:,:,np.newaxis], px_x[:, :, np.newaxis]], axis=-1)
pyx = pyx.astype(np.float32)

vud1 = np.dstack([pyx, depth]).reshape([-1, 3])

vud1 = vud1[vud1[:, 2] > 0]
uvd = vud1[:, (1, 0, 2)]


from psegs.util.plotting import draw_xy_depth_in_image
draw_xy_depth_in_image(debug, uvd, period_meters=0.1)


import torchvision.transforms.functional as F
import numpy as np
pil_img = F.to_pil_image(debug.astype(np.uint8))

from IPython.display import display
display(pil_img)



In [ ]:
# # Select the viewpoint using spherical angles  
# distance = 1.5   # distance from camera to the object
# elevation = 50.0   # angle of elevation in degrees
# azimuth = 0.0  # No rotation so the camera is positioned on the +Z axis. 

# # Get the position of the camera based on the spherical angles
# R, T = look_at_view_transform(distance, elevation, azimuth, device=device)
# print('R', R)
# print('T', T)

# cameras = FoVPerspectiveCameras(device=device, K=K, znear=0.1, zfar=100)

# raster_settings = RasterizationSettings(
#     image_size=256, 
#     blur_radius=0.0, 
#     faces_per_pixel=1, 
# )
# lights = PointLights(
#     device=device, 
#     location=[[0.0, 5.0, -10.0]], 
#     diffuse_color=((0, 0, 0),),
#     specular_color=((0, 0, 0),),
# )
# blend_params = BlendParams(sigma=1e-4, gamma=1e-4, background_color=(0.1, 0.1, 0.1))
# phong_renderer = MeshRenderer(
#     rasterizer=MeshRasterizer(
#         cameras=cameras, 
#         raster_settings=raster_settings
#     ),
#     shader=HardPhongShader(device=device, cameras=cameras, lights=lights, blend_params=blend_params)
# )


# image_ref = phong_renderer(meshes_world=teapot_mesh, R=R, T=T)
# print('image_ref', image_ref)

# import torchvision.transforms.functional as F
# import numpy as np
# pil_img = F.to_pil_image((255*image_ref.cpu().numpy()[0]).astype(np.uint8))

# from IPython.display import display
# display(pil_img)


In [ ]:
import pytorch3d.vis.plotly_vis
dir(pytorch3d.vis.plotly_vis)

In [2]:
import sys
sys.path.append('/opt/psegs')

import os

ROOT = '/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/'

for d in sorted(os.listdir(ROOT)):
    if '.DS_Store' in d:
        continue

    base_dir = os.path.join(ROOT, d)
    if not os.path.isdir(base_dir):
        print('skipping non-dir', base_dir)
        continue
    print()
    print()
    print()
    print(base_dir)
    

    outpath = os.path.join(ROOT, d + 'pytorch_rgbd_debug.mp4')
    if os.path.exists(outpath):
        print('aleady done', outpath)
        continue
    
    from psegs.datasets import ios_lidar


    from oarphpy import util as oputil
    json_paths = oputil.all_files_recursive(base_dir, pattern='frame*.json')
    json_paths = sorted(json_paths)
    
    try:
        cis = [ios_lidar.threeDScannerApp_create_camera_image(p) for p in json_paths]
    except AssertionError as e:
        print('err', e)
        continue

    print('len(cis)', len(cis))

    
    mesh_path = os.path.join(base_dir, 'export_refined.obj')
    if not os.path.exists(mesh_path):
        mesh_path = os.path.join(base_dir, 'export.obj')
    
    
    import imageio
    writer = imageio.get_writer(outpath, fps=5)
    
    from psegs.render.mesh2rgbd import pytorch3d_iter_mesh2uvd_for_camera_images
    
    iter_uvds = pytorch3d_iter_mesh2uvd_for_camera_images(cis, mesh_path, batch_size=2)
    for i, (ci, uvd) in enumerate(zip(cis, iter_uvds)):
        debug = ci.image
        from psegs.util.plotting import draw_xy_depth_in_image
        draw_xy_depth_in_image(debug, uvd, period_meters=0.1)
        writer.append_data(debug)
        print(i)
    
    writer.close()
    
    import torch
    torch.cuda.empty_cache()
    
    import gc
    gc.collect()
    
    print('done', outpath)





/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_31_01
err /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_31_01/frame_00001.jpg



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_55
err /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_55/frame_00001.jpg



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_36_35
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidar




/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_07_11
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_07_11pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_07_11.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_07_11pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_09_11
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_09_11pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_09_11.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_09_11pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_11_14
aleady done

aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_01_23pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_01_23.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_01_23pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_05_13
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_05_13pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_05_13.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_05_13pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_15_04
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_15_04pyt

skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_05_54.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_05_54pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_09_56
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_09_56pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_09_56.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_09_56pytorch_rgbd_debug.mp4



/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_25_46
aleady done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_25_46pytorch_rgbd_debug.mp4
skipping non-dir /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_25_

len(cis) 343
verts torch.Size([331421, 3])
faces torch.Size([583659, 3])
mesh <pytorch3d.structures.meshes.Meshes object at 0x7fb4dd041c10>
start batch
batch done 10.812266111373901
yielding 2
0
yielding 2
1
start batch
batch done 10.672507286071777
yielding 2
2
yielding 2
3
start batch
batch done 10.614536762237549
yielding 2
4
yielding 2
5
start batch
batch done 10.53771686553955
yielding 2
6
yielding 2
7
start batch
batch done 10.425113439559937
yielding 2
8
yielding 2
9
start batch
batch done 10.440918922424316
yielding 2
10
yielding 2
11
start batch
batch done 10.370381832122803
yielding 2
12
yielding 2
13
start batch
batch done 10.289090871810913
yielding 2
14
yielding 2
15
start batch
batch done 10.267695188522339
yielding 2
16
yielding 2
17
start batch
batch done 10.165318250656128
yielding 2
18
yielding 2
19
start batch
batch done 10.142006158828735
yielding 2
20
yielding 2
21
start batch
batch done 10.08971905708313
yielding 2
22
yielding 2
23
start batch
batch done 10.093392

...

103
start batch
batch done 9.615444421768188
yielding 2
104
yielding 2
105
start batch
batch done 9.590234994888306
yielding 2
106
yielding 2
107
start batch
batch done 9.759873628616333
yielding 2
108
yielding 2
109
start batch
batch done 9.795974969863892
yielding 2
110
yielding 2
111
start batch
batch done 9.807712078094482
yielding 2
112
yielding 2
113
start batch
batch done 9.774396657943726
yielding 2
114
yielding 2
115
start batch
batch done 9.803440570831299
yielding 2
116
yielding 2
117
start batch
batch done 9.771156549453735
yielding 2
118
yielding 2
119
start batch
batch done 9.844447612762451
yielding 2
120
yielding 2
121
start batch
batch done 9.818187475204468
yielding 2
122
yielding 2
123
start batch
batch done 9.808095216751099
yielding 2
124
yielding 2
125
start batch
batch done 9.786882162094116
yielding 2
126
yielding 2
127
start batch
batch done 9.749504089355469
yielding 2
128
yielding 2
129
start batch
batch done 9.76859188079834
yielding 2
130
yielding 2
131
sta

len(cis) 24
verts torch.Size([285274, 3])
faces torch.Size([533443, 3])
mesh <pytorch3d.structures.meshes.Meshes object at 0x7fb4d4f4b400>
start batch
batch done 9.860872268676758
yielding 2
0
yielding 2
1
start batch
batch done 9.77180814743042
yielding 2
2
yielding 2
3
start batch
batch done 9.790065288543701
yielding 2
4
yielding 2
5
start batch
batch done 9.80699634552002
yielding 2
6
yielding 2
7
start batch
batch done 9.652620792388916
yielding 2
8
yielding 2
9
start batch
batch done 9.410562515258789
yielding 2
10
yielding 2
11
start batch
batch done 9.328906059265137
yielding 2
12
yielding 2
13
start batch
batch done 9.339547872543335
yielding 2
14
yielding 2
15
start batch
batch done 9.230678081512451
yielding 2
16
yielding 2
17
start batch
batch done 9.366596460342407
yielding 2
18
yielding 2
19
start batch
batch done 9.247538566589355
yielding 2
20
yielding 2
21
start batch
batch done 9.203657150268555
yielding 2
22
yielding 2
23
done /outer_root/media/red14000/Pictures_and_

128
yielding 2
129
start batch
batch done 17.867539167404175
yielding 2
130
yielding 2
131
start batch
batch done 17.657208919525146
yielding 2
132
yielding 2
133
start batch
batch done 17.756351232528687
yielding 2
134
yielding 2
135
start batch
batch done 17.69210982322693
yielding 2
136
yielding 2
137
start batch
batch done 17.62552833557129
yielding 2
138
yielding 2
139
start batch
batch done 17.800055503845215
yielding 2
140
yielding 2
141
start batch
batch done 17.89986801147461
yielding 2
142
yielding 2
143
start batch
batch done 18.036298513412476
yielding 2
144
yielding 2
145
start batch
batch done 18.123709678649902
yielding 2
146
yielding 2
147
start batch
batch done 18.16549277305603
yielding 2
148
yielding 2
149
start batch
batch done 18.09208345413208
yielding 2
150
yielding 2
151
start batch
batch done 18.067283153533936
yielding 2
152
yielding 2
153
start batch
batch done 18.065958976745605
yielding 2
154
yielding 2
155
start batch
batch done 18.052181005477905
yielding

14
yielding 2
15
start batch
batch done 4.9071502685546875
yielding 2
16
yielding 2
17
start batch
batch done 4.902193546295166
yielding 2
18
yielding 2
19
start batch
batch done 4.910208225250244
yielding 2
20
yielding 2
21
start batch
batch done 4.886016368865967
yielding 2
22
yielding 2
23
start batch
batch done 4.903723955154419
yielding 2
24
yielding 2
25
start batch
batch done 4.868772506713867
yielding 2
26
yielding 2
27
start batch
batch done 4.867411851882935
yielding 2
28
yielding 2
29
start batch
batch done 4.887089014053345
yielding 2
30
yielding 2
31
start batch
batch done 4.928569555282593
yielding 2
32
yielding 2
33
start batch
batch done 4.94141411781311
yielding 2
34
yielding 2
35
start batch
batch done 4.953422784805298
yielding 2
36
yielding 2
37
start batch
batch done 4.966710567474365
yielding 2
38
yielding 2
39
start batch
batch done 4.962280750274658
yielding 2
40
yielding 2
41
start batch
batch done 4.933857202529907
yielding 2
42
yielding 2
43
start batch
batch

43
start batch
batch done 11.322331428527832
yielding 2
44
yielding 2
45
start batch
batch done 11.304376363754272
yielding 2
46
yielding 2
47
start batch
batch done 11.298699617385864
yielding 2
48
yielding 2
49
start batch
batch done 11.363690614700317
yielding 2
50
yielding 2
51
start batch
batch done 11.385607719421387
yielding 2
52
yielding 2
53
start batch
batch done 11.362175941467285
yielding 2
54
yielding 2
55
start batch
batch done 11.423774242401123
yielding 2
56
yielding 2
57
start batch
batch done 11.395496129989624
yielding 2
58
yielding 2
59
start batch
batch done 11.444361209869385
yielding 2
60
yielding 2
61
start batch
batch done 11.42470908164978
yielding 2
62
yielding 2
63
start batch
batch done 11.401618242263794
yielding 2
64
yielding 2
65
start batch
batch done 11.41335678100586
yielding 2
66
yielding 2
67
start batch
batch done 11.333601236343384
yielding 2
68
yielding 2
69
start batch
batch done 11.361776351928711
yielding 2
70
yielding 2
71
start batch
batch d